In [ ]:
import numpy as np
import json
import glob
from pprint import pprint
from collections import defaultdict
import os, math

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#vis
import pyLDAvis
import pyLDAvis.gensim

import BnLemma as lm
from bnlp import BengaliPOS




import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [21]:
anandabazar = "../corpus/txtFiles/anandabazar.txt"
bartaman = "../corpus/txtFiles/bartamanpatrika.txt"
banglaGanashakti = "../corpus/txtFiles/banglaGanashakti.txt"

In [22]:
# Bangla POS tagging
bn_pos = BengaliPOS()

from bnlp import BengaliCorpus as corpus
stopwords = corpus.stopwords[:20] + ['রি' + 'টি']
# stopwords[:10]

In [23]:
# def filter_doc(texts):
#     filtered = []
#     for line in texts:
#         filtered_line = [token for token in line if bn_pos.tag(token)[0][1] in ['NC','NP', 'JQ', 'JJ']] # 'NST',, 'AMN', , 'NCgen'
#         filtered_line = [token for token in filtered_line if token not in stopwords]
#         if len(filtered_line) > 0:
#             filtered.append(filtered_line)
                   
#     return filtered    

In [24]:
def lemmatize_bangla(text):
    if text is not None:
        lemma = lm.Lemmatizer()
        try:
            lemmatized = lemma.lemma(text)
            # print(lemmatized)
            return lemmatized.split()
        except Exception as e:
            # print(f"Error in lemmatizing text: {e}")
            return None



def filter_doc(texts):  
    filtered = []
    for line in texts:
            # print(line)
            
            # print(line)
            filtered_line = [token for token in line if bn_pos.tag(token)[0][1] in ['NC','NP', 'JQ', 'JJ']] # 'NST',, 'AMN', , 'NCgen'
            filtered_line = [token for token in filtered_line if token not in stopwords]
            line = lemmatize_bangla(' '.join(filtered_line))
            if line is not None:
                filtered.append(line)
                   
    return filtered

In [ ]:
data = open(banglaGanashakti, 'r')
lines = data.readlines()[:10000] # lecture par ligne
# print(lines[:10])
docs = [line.split() for line in lines]
docs[:10]

filtered_docs = filter_doc(docs)


# pprint(docs[:10])
# pprint(filtered_docs[:10])

for line, filtered_line in zip(docs[:10], filtered_docs[:10]):
    print(f'line = {line}, length = {len(line)}')
    print(f'filtered tokens = {filtered_line}, length = {len(filtered_line)}\n\n')

In [ ]:
# id2word
# id2word = corpora.Dictionary(docs) # docs = List[List[str]]
# corpus = [id2word.doc2bow(text) for text in filtered_docs]

# print(corpus)[0][:2]

In [ ]:
# ?
def gen_words(texts):
    data_words = [gensim.utils.simple_preprocess(text) for text in texts]
    return data_words

# single
single_string = [ ' '.join(line) for line in filtered_docs]
print(single_string[0][:100])
# for line in filtered_docs:
    
data_words  = gen_words(single_string)
print(data_words[9][:200])

In [ ]:
## BIGRAMMES ET TIGRAMMES

# filtered_docs ou data_words

bigram_phrases = gensim.models.Phrases(filtered_docs, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_docs], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

# bigram, trigram

def make_bigrams(texts):
    # print(bigram[doc] for doc in texts)
    return ([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])


data_bigrams = make_bigrams(filtered_docs)
data_bigrams_trigrams = make_trigrams(data_bigrams)

In [ ]:

print(data_bigrams_trigrams[0])

In [ ]:
### TF-IDF
from gensim.models import TfidfModel

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_docs]

print(corpus[0][:20])

tfidf = TfidfModel(corpus, id2word=id2word)
low_value = 0.03

words = []
words_missing_in_tfidf = []

# enlève les termes trops fréquents
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow



In [ ]:
# corpus = [id2word.doc2bow(text) for text in filtered_docs]
# print (corpus[0][0:20]) 

# word = id2word[[9][:1][0]]
# print (word)

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

#### Calcul spécificité

In [ ]:
def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False
    

def calculate_frequency(texts):
    frequency = defaultdict(int)    
    for text in texts:
        for token in text:
            if bn_pos.tag(token)[0][1] in ['NC','NP', 'JQ', 'JJ'] :
                frequency[token] +=1
                
    return dict(frequency)



def specif(f, F, t, T):
    try:
        r_command = f'R --vanilla -s -e \'library("textometry", lib="."); \
                    res <- specificities.distribution.plot({f},{F},{t},{T}); print(res["mode"]); \
                    print(res["pfsum"][[1]][[{f}+1]]);\''
        result = subprocess.check_output(r_command, shell=True, text=True).split('\n') # appel R
        mode, proba = str(result[1]).split()[1], str(result[3]).split()[1]
        if is_float(mode) and is_float(proba):
            mode, proba  = float(mode), float(proba)
            try:
                if mode <= f:
                    specificite = math.fabs(math.log10(math.fabs(proba))) 
                else :
                    specificite = -math.fabs(math.log10(math.fabs(proba))) 
                return specificite
            except ValueError:
                return None
    except subprocess.CalledProcessError as e:
            print(f" {e}.\nErreur en calculant la spécificité  pour : <{f}>, <{F}>, <{t}>, <{T}>")
            return None
    

## calcule f F t T -> appelle specif ##
def process_specif(focus_freq, window_freq, path):
    # if path exists else create it
    if not os.path.exists(path):
        os.makedirs(path)
    F = sum(focus_freq.values())
    T = sum(window_freq.values())
    spec_dict={}
    for key in focus_freq.keys():
        f = focus_freq[key]
        if key in window_freq.keys():
            t = window_freq[key]
        # on exlut les t=0
            specificite = specif(f, F, t, T) # specificite pour mot=key
            if specificite is not None:
                spec_dict[key] = specificite
                filename = path + f"/specif-{f}-{F}-{t}-{T}.txt" 
                with open(filename, 'w') as file:
                    file.write(f'{key}\t{specificite}\n')
        
    return spec_dict      



In [ ]:
# banglaGanashakti = '../corpus/txtFiles/banglaGanashakti.txt'
# anandabazar = '../corpus/txtFiles//anandabazar.txt'

data = open(anandabazar, 'r')
lines = data.readlines() # lecture par ligne
# print(lines[:10])
docs = [line.split() for line in lines]
docs[:10]
filtered_docs = filter_doc(docs)
focus_dict = calculate_frequency(filtered_docs[:len(filtered_docs)//4])
window_dict= calculate_frequency(filtered_docs[len(filtered_docs)//4:])

# print(focus_dict, len(focus_dict))
# print(window_dict, len(window_dict))
print(len(focus_dict), len(window_dict))
specificite = process_specif(focus_dict, window_dict, path="cache/") # dict specificite

# affiche max et min
max_values = sorted(specificite.items(), key = lambda x: x[1], reverse = True)[:20]
min_values = sorted(specificite.items(), key = lambda x: x[1])[:20]

print("\nMoins spécifiques :")
for sp in min_values:
    print(sp)
    
print("\nPlus spécifiques :")
for sp in sorted(max_values, key= lambda x : x[1]): 
    print(sp)


In [ ]:
# def calculate_frequency(text):
#     frequency = defaultdict(int)    
#     for token in doc:
#         if bn_pos.tag(token)[0][1] in ['NC','NP', 'JQ', 'JJ'] :
#             frequency[token] +=1
                
#     return dict(frequency)



In [ ]:
# def specif(f, F, t, T):
#     try:
#         r_command = f'R --vanilla -s -e \'library("textometry", lib="."); \
#                     res <- specificities.distribution.plot({f},{F},{t},{T}); print(res["mode"]); \
#                     print(res["pfsum"][[1]][[{f}+1]]);\''
#         result = subprocess.check_output(r_command, shell=True, text=True).split('\n') # appel R
#         mode, proba = str(result[1]).split()[1], str(result[3]).split()[1]
#         if is_float(mode) and is_float(proba):
#             mode, proba  = float(mode), float(proba)
#             try:
#                 if mode <= f:
#                     specificite = math.fabs(math.log10(math.fabs(proba))) 
#                 else :
#                     specificite = -math.fabs(math.log10(math.fabs(proba))) 
#                 return specificite
#             except ValueError:
#                 return None
#     except subprocess.CalledProcessError as e:
#             print(f" {e}.\nErreur en calculant la spécificité  pour : <{f}>, <{F}>, <{t}>, <{T}>")
#             return None
    

In [ ]:

# ## calcule f F t T -> appelle specif ##
# def process_specif(focus_freq, window_freq, path):
#     F = sum(focus_freq.values())
#     T = sum(window_freq.values())
#     spec_dict={}
#     for key in focus_freq.keys():
#         f = focus_freq[key]
#         if key in window_freq.keys():
#             t = window_freq[key]
#         # on exlut les t=0
#             specificite = specif(f, F, t, T) # specificite pour mot=key
#             if specificite is not None:
#                 spec_dict[key] = specificite
#                 filename = path + f"/specif-{f}-{F}-{t}-{T}.txt" 
#                 with open(filename, 'w') as file:
#                     file.write(f'{key}\t{specificite}\n')
        
#     return spec_dict               
  
        

In [ ]:
# focus_dict = (calculate_frequency(focus_corpus)) # focus_corpus = [
# window_dict = calculate_frequency(window_corpus)
# specificite = process_specif(focus_dict, window_dict, path="cache/") # dict specificite

# # affiche max et min
# max_values = sorted(specificite.items(), key = lambda x: x[1], reverse = True)[:10]
# min_values = sorted(specificite.items(), key = lambda x: x[1])[:10]

# print("\nMoins spécifiques :")
# for sp in min_values:
#     print(sp)
    
# print("\nPlus spécifiques :")
# for sp in sorted(max_values, key= lambda x : x[1]): 
#     print(sp)
